In [1]:
#NOTE: For the code to work, you must change the directory below in the glob.glob line

import requests
import pandas as pd
import numpy as np
import csv
import glob

In [2]:
#This only needs to be executed once, then the files are on your local

yr = 2017

while yr > 1989:
    url = f'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_cbsa_{yr}.zip'
    response = requests.get(url)
    yr -= 1
    with open('temp.zip', 'wb') as f:
        f.write(response.content)
        !unzip 'temp.zip'

Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2017.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2016.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2015.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2014.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2013.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2012.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2011.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2010.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2009.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2008.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2007.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2006.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2005.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2004.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2003.csv  
Archive:  temp.zip
  inflating: daily_aqi_by_cbsa_2002.csv  
Archive:  temp.zip
  inf

In [32]:
df = pd.DataFrame()
x = 27

for file in glob.glob('C:/Users/dshaf/Documents/School/classproject/ProjectOne/Project Data/daily*'):

    data = pd.read_csv(file, header = 0)
    df = df.append(data)
    print('file: ' + str(file[-8:-4]) + ', ' + str(x) + ' to go')
    x -= 1
    
atlanta_df = df.loc[  (df['CBSA'].apply( lambda s:  "Atlanta" in s  )) & (df['Defining Parameter'] == 'Ozone')]
dallas_df = df.loc[  (df['CBSA'].apply( lambda s:  "Dallas" in s  )) & (df['Defining Parameter'] == 'Ozone')]
austin_df = df.loc[  (df['CBSA'].apply( lambda s:  "Austin" in s  )) & (df['Defining Parameter'] == 'Ozone')]
nashville_df = df.loc[  (df['CBSA'].apply( lambda s:  "Nashville" in s  )) & (df['Defining Parameter'] == 'Ozone')]
combined_df = df.loc[((df['CBSA'].apply( lambda s:  "Atlanta" in s  )) | (df['CBSA'].apply( lambda s:  "Dallas" in s  )) | (df['CBSA'].apply( lambda s:  "Nashville" in s  )) | (df['CBSA'].apply( lambda s:  "Austin" in s  ))) & (df['Defining Parameter'] == 'Ozone')]

file: 1990, 27 to go
file: 1991, 26 to go
file: 1992, 25 to go
file: 1993, 24 to go
file: 1994, 23 to go
file: 1995, 22 to go
file: 1996, 21 to go
file: 1997, 20 to go
file: 1998, 19 to go
file: 1999, 18 to go
file: 2000, 17 to go
file: 2001, 16 to go
file: 2002, 15 to go
file: 2003, 14 to go
file: 2004, 13 to go
file: 2005, 12 to go
file: 2006, 11 to go
file: 2007, 10 to go
file: 2008, 9 to go
file: 2009, 8 to go
file: 2010, 7 to go
file: 2011, 6 to go
file: 2012, 5 to go
file: 2013, 4 to go
file: 2014, 3 to go
file: 2015, 2 to go
file: 2016, 1 to go
file: 2017, 0 to go
